# Recommender on South Carolina Food Reviewers

The goal of this notebook is to build a rudimentary recomender system for South Carolina Reviewers

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data = pd.read_csv("food_reviews_sc")